In [2]:
from utils_corpus import *
import os

/home/ecoazul/alvaro/CORPUS/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Pregunta 1.  Listado de adjetivos en el texto con conteo y frecuencia.



## Extraccion de texto

In [4]:
input_folder = "/home/ecoazul/alvaro/CORPUS/data/input/corpus"
output_folder = "/home/ecoazul/alvaro/CORPUS/data/output/question1/1.frases"

split_sentences_dict = {}

for filename in tqdm(os.listdir(input_folder), desc="Procesando PDFs"):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(input_folder, filename)
        extracted_text = extract_pdf_text(pdf_path)
        cleaned_text = clean_text(extracted_text)
        output_path = os.path.join(
            output_folder,
            f"frases_{os.path.splitext(filename)[0]}.txt"
        )
        split_sentences = save_sentences_to_file(cleaned_text, output_path)
        split_sentences_dict[filename] = split_sentences

Procesando PDFs: 100%|██████████| 53/53 [00:35<00:00,  1.51it/s]


## Metodo Diccionario

In [6]:
from glob import glob

dic_folder = "/home/ecoazul/alvaro/CORPUS/data/input/dic"
# Obtener todos los archivos de frases
frases_folder = "/home/ecoazul/alvaro/CORPUS/data/output/question1/1.frases"
frases_files = glob(os.path.join(frases_folder, "frases_*.txt"))
output_folder = "/home/ecoazul/alvaro/CORPUS/data/output/question1/2.Adjetivos diccionario"
os.makedirs(output_folder, exist_ok=True)
resultados = {}

for frases_path in frases_files:
    # Extraer el idioma del nombre del archivo (ejemplo: frases_01_ES_Cicloturismo.txt -> es)
    partes = os.path.basename(frases_path).split("_")
    if len(partes) >= 3:
        idioma = partes[2].lower()
        dic_path = os.path.join(dic_folder, f"{idioma}.adj")
        if os.path.exists(dic_path):
            # Cargar diccionario de adjetivos
            adjectives_map = load_adjective_dictionary(dic_path)
            # Leer frases
            with open(frases_path, encoding="utf-8") as f:
                split_sentences = [line.strip() for line in f if line.strip()]
            # Extraer adjetivos
            df_adjectives = extract_adjectives_from_text(split_sentences, adjectives_map)
            resultados[frases_path] = df_adjectives
            # Guardar el DataFrame de adjetivos en un CSV
            csv_output = output_folder + "/" + os.path.basename(frases_path).replace(".txt", "_adjetivos.csv")
            df_adjectives.to_csv(csv_output, index=False, encoding="utf-8", sep=";")
        else:
            print(f"No se encontró diccionario para el idioma: {idioma} en {frases_path}")
    else:
        print(f"Nombre de archivo inesperado: {frases_path}")

No se encontró diccionario para el idioma: po en /home/ecoazul/alvaro/CORPUS/data/output/question1/1.frases/frases_05_PO_Naturaleza.txt
No se encontró diccionario para el idioma: jp en /home/ecoazul/alvaro/CORPUS/data/output/question1/1.frases/frases_03_JP_Verde.txt
No se encontró diccionario para el idioma: zh en /home/ecoazul/alvaro/CORPUS/data/output/question1/1.frases/frases_02_ZH_Tren.txt
No se encontró diccionario para el idioma: jp en /home/ecoazul/alvaro/CORPUS/data/output/question1/1.frases/frases_05_JP_Naturaleza.txt
No se encontró diccionario para el idioma: zh en /home/ecoazul/alvaro/CORPUS/data/output/question1/1.frases/frases_05_ZH_Naturaleza.txt
No se encontró diccionario para el idioma: ko en /home/ecoazul/alvaro/CORPUS/data/output/question1/1.frases/frases_03_KO_Verde.txt
No se encontró diccionario para el idioma: zh en /home/ecoazul/alvaro/CORPUS/data/output/question1/1.frases/frases_03_ZH_Verde.txt
No se encontró diccionario para el idioma: jp en /home/ecoazul/alvaro

### Frecuency

In [8]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Procesa TODOS los pares CSV/TXT de las carpetas:
    2.Adjetivos diccionario   (CSV con _adjetivos.csv)
    1.frases copy             (TXT completo)

Para cada par genera un CSV en 3.Porcentaje adj dic con:
    adjetivo_base, variantes, frecuencia, porcentaje
"""

from pathlib import Path
import re
from collections import defaultdict
import pandas as pd

# ── 1. Carpetas ────────────────────────────────────────────────────────────
ROOT = Path("/home/ecoazul/alvaro/CORPUS/data/output/question1")

CSV_DIR   = ROOT / "2.Adjetivos diccionario"
TXT_DIR   = ROOT / "1.frases"
OUT_DIR   = ROOT / "3.Porcentaje adj dic"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ── 2. Funciones auxiliares ────────────────────────────────────────────────
def limpia(token: str) -> str:
    """Quita espacios y pasa a minúsculas."""
    return token.strip().lower()

def construir_diccionarios(csv_path: Path):
    """Devuelve (variant_to_base, base_to_variants) a partir del CSV."""
    df = pd.read_csv(csv_path, sep=";", dtype=str).fillna("")
    variant_to_base = {}
    base_to_variants = defaultdict(set)

    for _, row in df.iterrows():
        variants = [limpia(v) for v in row["adjective_variant"].split(",") if limpia(v)]
        bases    = [limpia(b) for b in row["adjective_base"].split(",")   if limpia(b)]

        if not variants and not bases:
            continue

        if len(variants) == len(bases) and bases:
            pairs = zip(variants, bases)
        elif len(bases) == 1:
            pairs = [(v, bases[0]) for v in variants]
        else:
            bases_set = set(bases)
            pairs = [(v, v) for v in variants if v in bases_set]

        for v, b in pairs:
            variant_to_base.setdefault(v, b)
            if v != b:
                base_to_variants[b].add(v)

        for b in bases:
            variant_to_base.setdefault(b, b)

    return variant_to_base, base_to_variants

def procesar_par(csv_path: Path, txt_path: Path, out_path: Path):
    """Genera el CSV de frecuencias/porcentaje para un par concreto."""
    variant_to_base, base_to_variants = construir_diccionarios(csv_path)

    texto = txt_path.read_text(encoding="utf-8").lower()
    tokens = re.findall(r"[a-záéíóúüñ]+", texto, flags=re.IGNORECASE)
    total_palabras = len(tokens)

    frecuencia_base = defaultdict(int)
    for tok in tokens:
        base = variant_to_base.get(tok)
        if base:
            frecuencia_base[base] += 1

    filas = []
    for base, freq in sorted(frecuencia_base.items(), key=lambda x: x[1], reverse=True):
        if freq == 0:
            continue
        variantes = ", ".join(sorted(base_to_variants[base]))
        porcentaje = round(freq / total_palabras * 100, 2)
        filas.append(
            {
            "adjetivo_base": base,
            "variantes": variantes,
            "frecuencia": freq,
            "porcentaje": f"{porcentaje}%",
            }
        )

    pd.DataFrame(filas).to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"[✓] {out_path.name} generado con {len(filas)} adjetivos (total {total_palabras} palabras)")

# ── 3. Bucle principal ─────────────────────────────────────────────────────
for csv_file in CSV_DIR.glob("*_adjetivos.csv"):
    # Ej.: frases_01_ES_Cicloturismo_adjetivos.csv → base = frases_01_ES_Cicloturismo
    base_name = csv_file.stem.replace("_adjetivos", "")
    txt_file  = TXT_DIR / f"{base_name}.txt"

    if not txt_file.exists():
        print(f"[!] TXT no encontrado para {csv_file.name}, se omite.")
        continue

    out_file = OUT_DIR / f"{base_name}_adj_freq.csv"
    procesar_par(csv_file, txt_file, out_file)


[✓] frases_02_FR_Tren_adj_freq.csv generado con 198 adjetivos (total 5289 palabras)
[✓] frases_02_EN_Tren_adj_freq.csv generado con 210 adjetivos (total 5144 palabras)
[✓] frases_07_FR_Turismo sostenible_adj_freq.csv generado con 263 adjetivos (total 6069 palabras)
[✓] frases_01_ES_Cicloturismo_adj_freq.csv generado con 258 adjetivos (total 5834 palabras)
[✓] frases_04_ES_Gran Ruta Verde_adj_freq.csv generado con 459 adjetivos (total 13898 palabras)
[✓] frases_03_IT_Verde_adj_freq.csv generado con 314 adjetivos (total 5313 palabras)
[✓] frases_08_EN_Turismo rural_adj_freq.csv generado con 249 adjetivos (total 6901 palabras)
[✓] frases_01_FR_Cicloturismo_adj_freq.csv generado con 199 adjetivos (total 6388 palabras)
[✓] frases_02_IT_Tren_adj_freq.csv generado con 290 adjetivos (total 4950 palabras)
[✓] frases_02_RU_Tren_adj_freq.csv generado con 0 adjetivos (total 209 palabras)
[✓] frases_04_EN_Gran Ruta Verde_adj_freq.csv generado con 365 adjetivos (total 14409 palabras)
[✓] frases_06_E

## Metodo IA

In [ ]:
import os
from glob import glob
from tqdm import tqdm

MODEL ='mistral-small:latest'

frases_folder = "/home/ecoazul/alvaro/CORPUS/data/output/question1/1.frases copy"
output_folder = "/home/ecoazul/alvaro/CORPUS/data/output/question1/4.Adjetivos ia"
dic_folder = "/home/ecoazul/alvaro/CORPUS/data/input/dic"
os.makedirs(output_folder, exist_ok=True)

frases_files = glob(os.path.join(frases_folder, "frases_*.txt"))

for frases_path in tqdm(frases_files, desc="Procesando archivos", position=0, leave=True):
    with open(frases_path, encoding="utf-8") as f:
        split_sentences = [line.strip() for line in f if line.strip()]
    lenguage = os.path.basename(frases_path).split("_")[2].lower()
    dic_path = os.path.join(dic_folder, f"{lenguage}.adj")
    adjectives_map = load_adjective_dictionary(dic_path)
    df = process_sentences_with_ollama(split_sentences, MODEL, lenguage, adjectives_map,max_retries=50)
    output_csv = os.path.join(output_folder, os.path.basename(frases_path).replace(".txt", "_ia.csv"))
    df.to_csv(output_csv, index=False, encoding="utf-8", sep=";")
    print(f"Procesado y guardado: {output_csv}")

### Frecuency ia_variant

In [ ]:
import os
from tqdm import tqdm
from collections import defaultdict
from pathlib import Path
import pandas as pd
ROOT = Path("/home/ecoazul/alvaro/CORPUS/data/output/question1")

TXT_DIR   = ROOT / "1.frases copy"
CSV_DIR   = ROOT / "4.Adjetivos ia"
OUT_DIR = ROOT / "5.porcentaje adj ia variant"
OUT_DIR.mkdir(parents=True, exist_ok=True)

frequencies = defaultdict(int)
for csv_file in tqdm(CSV_DIR.glob("*ia.csv"), desc="Procesando CSVs"):
    csv = pd.read_csv(csv_file, sep=";", dtype=str).fillna("")
    frequencies = defaultdict(int)
    for _,fila in csv.iterrows():
        for adj in fila["Adjectives_IA_variant"].split(","):
            adjetivo = adj.strip().lower()
            frequencies[adjetivo] += 1
    # Guardar el resultado de frecuencias en un CSV
    resultados = pd.DataFrame([
        {"adjetivo": adj, "frecuencia": freq}
        for adj, freq in frequencies.items() if adj and adj != "none"
    ])
    resultados = resultados.sort_values(by="frecuencia", ascending=False)
    output_csv = OUT_DIR / (csv_file.stem.replace("_ia", "") + "_adj_freq_ia.csv")
    resultados.to_csv(output_csv, index=False, encoding="utf-8-sig")

    # Calcular el total de palabras en el texto original correspondiente
    base_name = csv_file.stem.replace("_ia", "")
    txt_file = TXT_DIR / f"{base_name}.txt"
    if txt_file.exists():
        with open(txt_file, encoding="utf-8") as f:
            texto = f.read().lower()
        tokens = re.findall(r"[a-záéíóúüñ]+", texto, flags=re.IGNORECASE)
        total_palabras = len(tokens)
        # Añadir columna de porcentaje
        resultados["porcentaje"] = round(resultados["frecuencia"] / total_palabras * 100, 2).astype(str) + "%"
        resultados.to_csv(output_csv, index=False, encoding="utf-8-sig")
        print(f"Guardado: {output_csv}")

    else:
        print(f"Archivo de texto no encontrado: {txt_file}")


### Frecuency ia base

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Procesa TODOS los pares CSV/TXT de las carpetas:
    2.Adjetivos diccionario   (CSV con _adjetivos.csv)
    1.frases copy             (TXT completo)

Para cada par genera un CSV en 3.Porcentaje adj dic con:
    adjetivo_base, variantes, frecuencia, porcentaje
"""

from pathlib import Path
import re
from collections import defaultdict
import pandas as pd

# ── 1. Carpetas ────────────────────────────────────────────────────────────
ROOT = Path("/home/ecoazul/alvaro/CORPUS/data/output/question1")

CSV_DIR   = ROOT / "4.Adjetivos ia"
TXT_DIR   = ROOT / "1.frases copy"
OUT_DIR   = ROOT / "6.Porcentaje adj ia base"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ── 2. Funciones auxiliares ────────────────────────────────────────────────
def limpia(token: str) -> str:
    """Quita espacios y pasa a minúsculas."""
    return token.strip().lower()

def construir_diccionarios(csv_path: Path):
    """Devuelve (variant_to_base, base_to_variants) a partir del CSV."""
    df = pd.read_csv(csv_path, sep=";", dtype=str).fillna("")
    variant_to_base = {}
    base_to_variants = defaultdict(set)

    for _, row in df.iterrows():
        variants = [limpia(v) for v in row["Adjectives_IA_variant"].split(",") if limpia(v)]
        bases    = [limpia(b) for b in row["Adjectives_IA_base"].split(",")   if limpia(b)]

        if not variants and not bases:
            continue

        if len(variants) == len(bases) and bases:
            pairs = zip(variants, bases)
        elif len(bases) == 1:
            pairs = [(v, bases[0]) for v in variants]
        else:
            bases_set = set(bases)
            pairs = [(v, v) for v in variants if v in bases_set]

        for v, b in pairs:
            variant_to_base.setdefault(v, b)
            if v != b:
                base_to_variants[b].add(v)

        for b in bases:
            variant_to_base.setdefault(b, b)

    return variant_to_base, base_to_variants

def procesar_par(csv_path: Path, txt_path: Path, out_path: Path):
    """Genera el CSV de frecuencias/porcentaje para un par concreto."""
    variant_to_base, base_to_variants = construir_diccionarios(csv_path)

    texto = txt_path.read_text(encoding="utf-8").lower()
    tokens = re.findall(r"[a-záéíóúüñ]+", texto, flags=re.IGNORECASE)
    total_palabras = len(tokens)

    frecuencia_base = defaultdict(int)
    for tok in tokens:
        base = variant_to_base.get(tok)
        if base:
            frecuencia_base[base] += 1

    filas = []
    for base, freq in sorted(frecuencia_base.items(), key=lambda x: x[1], reverse=True):
        if freq == 0:
            continue
        variantes = ", ".join(sorted(base_to_variants[base]))
        porcentaje = round(freq / total_palabras * 100, 2)
        filas.append(
            {
            "adjetivo_base": base,
            "variantes": variantes,
            "frecuencia": freq,
            "porcentaje": f"{porcentaje}%",
            }
        )

    pd.DataFrame(filas).to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"[✓] {out_path.name} generado con {len(filas)} adjetivos (total {total_palabras} palabras)")

# ── 3. Bucle principal ─────────────────────────────────────────────────────
for csv_file in CSV_DIR.glob("*_ia.csv"):
    # Ej.: frases_01_ES_Cicloturismo_adjetivos.csv → base = frases_01_ES_Cicloturismo
    base_name = csv_file.stem.replace("_ia", "")
    txt_file  = TXT_DIR / f"{base_name}.txt"

    if not txt_file.exists():
        print(f"[!] TXT no encontrado para {csv_file.name}, se omite.")
        continue

    out_file = OUT_DIR / f"{base_name}_adj_freq_ia_base.csv"
    procesar_par(csv_file, txt_file, out_file)


## Resultados

In [9]:
import os
import pandas as pd

def comparar_adjetivos(row):
    # Lee los dos campos, trata NaN como cadena vacía
    val1 = "" if pd.isna(row["adjective_base"])       else str(row["adjective_base"])
    val2 = "" if pd.isna(row["Adjectives_IA_base"])   else str(row["Adjectives_IA_base"])
    
    # Divide, quita espacios y filtra los indeseables
    filtro = lambda w: w and w.lower() not in ("none", "nan")
    set1 = {w.strip() for w in val1.split(",") if filtro(w.strip())}
    set2 = {w.strip() for w in val2.split(",") if filtro(w.strip())}
    
    # Si ambos están vacíos, no pongas nada
    if not set1 and not set2:
        return ""
    
    # Calcula comunes y no comunes
    comunes   = sorted(set1 & set2)
    solo_dic  = sorted(set1 - set2)
    solo_ia   = sorted(set2 - set1)
    
    # Junta solo las partes no vacías, separadas por coma
    partes = []
    if comunes:   partes.append(", ".join(comunes))
    if solo_dic:  partes.append(", ".join(solo_dic))
    if solo_ia:   partes.append(", ".join(solo_ia))
    
    return ", ".join(partes)

union_folder = "/home/ecoazul/alvaro/CORPUS/data/output/question1/7.union"
os.makedirs(union_folder, exist_ok=True)

diccionario_folder = "/home/ecoazul/alvaro/CORPUS/data/output/question1/2.Adjetivos diccionario"
ia_folder         = "/home/ecoazul/alvaro/CORPUS/data/output/question1/4.Adjetivos ia"

diccionario_files = sorted(f for f in os.listdir(diccionario_folder) if f.endswith("_adjetivos.csv"))
ia_files         = sorted(f for f in os.listdir(ia_folder)       if f.endswith("_ia.csv"))

for dic_file, ia_file in zip(diccionario_files, ia_files):
    df_dic = pd.read_csv(os.path.join(diccionario_folder, dic_file),
                         sep=";", encoding="utf-8")
    df_ia  = pd.read_csv(os.path.join(ia_folder, ia_file),
                         sep=";", encoding="utf-8")

    if "sentence" in df_dic.columns and "sentence" in df_ia.columns:
        df_union = df_dic.merge(
            df_ia[["sentence", "Adjectives_IA_base"]],
            on="sentence", how="inner"
        )[
            ["sentence", "adjective_base", "Adjectives_IA_base"]
        ]

        # Aplica la función ya limpia de valores vacíos
        df_union["adjectives"] = df_union.apply(comparar_adjetivos, axis=1)

        output_path = os.path.join(
            union_folder,
            dic_file.replace("_adjetivos.csv", "_union.csv")
        )
        df_union.to_csv(output_path, index=False, encoding="utf-8", sep=";")
    else:
        print(f"Column 'sentence' not found in: {dic_file}, {ia_file}")


# Pregunta 2. Listado adjetivos-sustantivos.

## Procesamiento IA

In [5]:
import os
import pandas as pd

MODEL = 'mistral-small:latest'
input_folder = '/home/ecoazul/alvaro/CORPUS/data/output/question1/7.union'
output_folder = '/home/ecoazul/alvaro/CORPUS/data/output/question2/1.adjetivos_sustantivos'
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        input_csv = os.path.join(input_folder, filename)
        output_file = os.path.join(output_folder, filename.replace('.csv', '.xlsx'))
        df = pd.read_csv(input_csv, sep=';')
        result_df = process_nouns(df, model=MODEL, output_file=output_file)

Processing adjectives: 100%|██████████| 106/106 [00:31<00:00,  3.40it/s]


## Conteo

In [9]:
df_adj_to_sentence = pd.read_csv('/home/ecoazul/alvaro/CORPUS/data/output/question2/1.adjetivos_sustantivos/frases_01_ES_Cicloturismo_union.xlsx', sep=';')
# Suponiendo que ya tienes el DataFrame 'df'
agrupado = df_adj_to_sentence.groupby('adjective')['noun'].apply(lambda x: ', '.join(x.unique())).reset_index()

# Crear una nueva columna con el conteo de sustantivos
agrupado['Conteo'] = agrupado['noun'].apply(lambda x: len(x.split(', ')))

# Ordenar el DataFrame según el conteo de sustantivos de forma descendente (o ascendente si lo prefieres)
agrupado = agrupado.sort_values(by='Conteo', ascending=False)

# Guardar el resultado en un CSV en la ruta especificada
agrupado.to_csv('/home/ecoazul/alvaro/CORPUS/data/output/question2/conteo.csv', index=False)

# Pregunta 3. Porcentaje de colores.

## Imagenes a PDF

In [ ]:
import os
from tqdm import tqdm 

input_folder = "/home/ecoazul/alvaro/CORPUS/data/input/corpus"
output_folder = "/home/ecoazul/alvaro/CORPUS/data/output/question3/1.pdf_complete_to_images"

for filename in tqdm(os.listdir(input_folder),desc="Procesando PDFs"):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(input_folder, filename)
        # Crear carpeta específica para cada PDF (sin extensión)
        pdf_name = os.path.splitext(filename)[0]
        pdf_output_folder = os.path.join(output_folder, pdf_name)
        os.makedirs(pdf_output_folder, exist_ok=True)
        print(f"Procesando: {pdf_path} -> {pdf_output_folder}")
        total_pages = convert_pdf_to_images(pdf_path, pdf_output_folder)
        print(f"{filename}: {total_pages} páginas convertidas.")

## Analisis por paginas

In [ ]:

# Rutas de entrada y salida
input_folder = "/home/ecoazul/alvaro/CORPUS/data/output/question3/1.pdf_complete_to_images"
output_folder = "/home/ecoazul/alvaro/CORPUS/data/output/question3/2.analisis_por_paginas"
num_colors = 10

process_folder(input_folder, output_folder, num_colors)



## Analisis completo - miniBatchKMeans

In [5]:
input_folder = "/home/ecoazul/alvaro/CORPUS/data/output/question3/1.pdf_complete_to_images"
output_folder = "/home/ecoazul/alvaro/CORPUS/data/output/question3/3.analisis_completo"
num_colors = 5

complete_analysis(input_folder, output_folder, num_colors)

Procesando PDFs completos:   0%|          | 0/1 [00:00<?, ?it/s]


01_ES_Cicloturismo: Detectadas 1 páginas para extracción de color dominante.


Procesando PDFs completos: 100%|██████████| 1/1 [00:00<00:00,  6.06it/s]

PDF generado: /home/ecoazul/alvaro/CORPUS/data/output/question3/3.analisis_completo/01_ES_Cicloturismo/overall_dominant_colors.pdf


# Pregunta 4. Analisis de imagenes

## Extraccion de imagenes

In [ ]:
from utils_corpus import *
import os

input_folder = "/home/ecoazul/alvaro/CORPUS/data/input/corpus"
output_base_folder = "/home/ecoazul/alvaro/CORPUS/data/output/question4/1.images_from_pdf"

os.makedirs(output_base_folder, exist_ok=True)

for filename in tqdm(os.listdir(input_folder),desc="Obteniendo imagenes de PDFs"):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(input_folder, filename)
        pdf_name = os.path.splitext(filename)[0]
        output_folder = os.path.join(output_base_folder, pdf_name)
        os.makedirs(output_folder, exist_ok=True)
        images = extract_images_from_pdf(pdf_path, output_folder)

## Modelos Captions 

### BLIP-2 + Flan-T5

In [2]:
import os

input_folder = '/home/ecoazul/alvaro/CORPUS/data/output/question4/1.images_from_pdf'
output_base = '/home/ecoazul/alvaro/CORPUS/data/output/question4/2.description'
model_name = "Salesforce/instructblip-flan-t5-xxl"

# Cargar modelo y procesador
processor, model, device = load_model_instruct(model_name)

os.makedirs(output_base, exist_ok=True)
for root, dirs, files in os.walk(input_folder):
    image_files = [os.path.join(root, f) for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if not image_files:
        continue

    # Nombre base para el CSV (usa solo el nombre de la carpeta relativa)
    folder_name = os.path.relpath(root, input_folder)
    output_csv = os.path.join(output_base, f"{folder_name.replace(os.sep, '_')}_description.csv")
    results = process_images_in_folder(root, processor, model, device, output_csv)
    save_results_to_csv(results, output_csv)


Loading checkpoint shards: 100%|██████████| 5/5 [00:26<00:00,  5.29s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
Procesando imágenes: 100%|██████████| 2/2 [09:23<00:00, 281.57s/it]


## Procesamiento IA Generativa - elementos imagenes(desde descripcion)

In [3]:
import os
import pandas as pd
from tqdm import tqdm

input_folder = '/home/ecoazul/alvaro/CORPUS/data/output/question4/2.description'
output_folder = '/home/ecoazul/alvaro/CORPUS/data/output/question4/3.elementos'
os.makedirs(output_folder, exist_ok=True)
MODEL = 'mistral-small:latest'

for filename in tqdm(os.listdir(input_folder),desc="Procesando descripciones"):
    if filename.endswith('.csv'):
        input_csv = os.path.join(input_folder, filename)
        df = pd.read_csv(input_csv)
        df['elementos'] = df['Description'].apply(lambda s: process_description(s, MODEL))
        output_csv = os.path.join(output_folder, filename)
        df.to_csv(output_csv, index=False)

Procesando descripciones: 100%|██████████| 1/1 [00:01<00:00,  1.73s/it]


## Conteo


In [2]:
process_folder_elem(
    '/home/ecoazul/alvaro/CORPUS/data/output/question4/3.elementos',
    '/home/ecoazul/alvaro/CORPUS/data/output/question4/4.conteo',
    column_name='elementos'
)

Guardado: /home/ecoazul/alvaro/CORPUS/data/output/question4/4.conteo/01_ES_Cicloturismo_description.docx
